# Parameter Indexing

For the foreseeable future, parameter indexing is outside of ParamTools' scope. However, indexed parameters are an important part of many modeling projects. Let's take a look at how one might implement indexed parameters with ParamTools.

Setup
--------------------------

The TaxParams parameters will be used to demonstrate parameter indexing. These parameters are based directly on the Tax-Calculator policy parameters. Tax-Calculator does really serious parameter indexing. It blows up values using inflation rates and wage growth rates. This tutorial demonstrates how you can replicate that same level of parameter indexing with ParamTools.

The approach for this tutorial is to build a `IndexedParameters` class on top of `paramtools.Parameters`. This class can then be used by projects that require parameter indexing.

### Get the code and the data

1. Clone the ParamTools repo: https://github.com/PSLmodels/ParamTools
2. Install paramtools
    ```
    conda create -n paramtools-env numpy taxcalc pip -c pslmodels
    conda activate paramtools-env
    pip install paramtools
    ```
3. Change directories to `paramtools/examples/taparams-demo`



In [1]:
# quick helper to print stuff out nicely.
def pprint(vals):
    for v in vals:
        print(v)

### TaxParams

Before we get started, let's make sure that the TaxParams can be loaded.

In [2]:
from marshmallow import Schema, fields

import paramtools

# first define the compatible data custom field.
class CompatibleDataSchema(Schema):
    """
    Schema for Compatible data object
    {
        "compatible_data": {"data1": bool, "data2": bool, ...}
    }
    """

    puf = fields.Boolean()
    cps = fields.Boolean()

class TaxParams(paramtools.Parameters):
    # You need to be in the paramtools/examples/taxparams directory!
    schema = "schema.json"
    defaults = "defaults.json"
    field_map = {"compatible_data": fields.Nested(CompatibleDataSchema())}

params = TaxParams()
print("EITC celing max year: ", max(map(lambda x: x["year"], params._EITC_c)), "\n")
print("EITC ceiling ceiling as dict: ")
pprint(params._EITC_c)

EITC celing max year:  2018 

EITC ceiling ceiling as dict: 
{'EIC': '0kids', 'value': 487.0, 'year': 2013}
{'EIC': '1kid', 'value': 3250.0, 'year': 2013}
{'EIC': '2kids', 'value': 5372.0, 'year': 2013}
{'EIC': '3+kids', 'value': 6044.0, 'year': 2013}
{'EIC': '0kids', 'value': 496.0, 'year': 2014}
{'EIC': '1kid', 'value': 3305.0, 'year': 2014}
{'EIC': '2kids', 'value': 5460.0, 'year': 2014}
{'EIC': '3+kids', 'value': 6143.0, 'year': 2014}
{'EIC': '0kids', 'value': 503.0, 'year': 2015}
{'EIC': '1kid', 'value': 3359.0, 'year': 2015}
{'EIC': '2kids', 'value': 5548.0, 'year': 2015}
{'EIC': '3+kids', 'value': 6242.0, 'year': 2015}
{'EIC': '0kids', 'value': 506.0, 'year': 2016}
{'EIC': '1kid', 'value': 3373.0, 'year': 2016}
{'EIC': '2kids', 'value': 5572.0, 'year': 2016}
{'EIC': '3+kids', 'value': 6269.0, 'year': 2016}
{'EIC': '0kids', 'value': 510.0, 'year': 2017}
{'EIC': '1kid', 'value': 3400.0, 'year': 2017}
{'EIC': '2kids', 'value': 5616.0, 'year': 2017}
{'EIC': '3+kids', 'value': 6318.0

Extend Parameters
------------------------

To get started, the ability to extend parameters through a given year needs to be added to ParamTools. This is done by finding the maximum specifed year for each parameter and duplicating each value object defined at this maximum year for the remaining years. For example, the maximum defined year for `_EITC_c` is 2018. There are four values defined in 2018. These four values will be extended for the remaining years, 2019 to 2028.

```json
[
    {"year": 2018, "value": 519.0, "EIC": "0kids"}, 
    {"year": 2018, "value": 3461.0, "EIC": "1kid"}, 
    {"year": 2018, "value": 5716.0, "EIC": "2kids"}, 
    {"year": 2018, "value": 6431.0, "EIC": "3+kids"}
]
```

In [34]:
from collections import defaultdict

import paramtools

class ExtendParameters(paramtools.Parameters):
    
    def extend(self):
        """
        Guarantee that all parameters are defined for each year
        from start year to end year.
        """
        max_allowed_year = max(self._stateless_dim_mesh["year"])
        adjustment = defaultdict(list)
        for param, data in self.specification(meta_data=True).items():
            max_year = max(map(lambda x: x["year"], data["value"]))
            if max_year == max_allowed_year:
                continue
            value_objects = self._get(param, True, year=max_year)
            while max_year < max_allowed_year:
                max_year += 1
                for value_object in value_objects:
                    adjustment[param].append(dict(value_object, **{"year": max_year}))
        self.adjust(adjustment)


class TaxParams(ExtendParameters):
    schema = "schema.json"
    defaults = "defaults.json"
    field_map = {"compatible_data": fields.Nested(CompatibleDataSchema())}

params = TaxParams()
params.extend()
print("EITC celing max year: ", max(map(lambda x: x["year"], params._EITC_c)), "\n")

EITC celing max year:  2028 



Since all parameters have been extended across the year axis, we can activate `array_first` mode. This means that all parameter values will be available as arrays instead of a list of dictionaries, or [value-objects][1]. 


[1]: https://paramtools.readthedocs.io/en/latest/spec.html#value-object

In [4]:
# all parameters can now be converted to arrays.
params.array_first=True
params.set_state()
print("EITC celing as array: ")
print(params._EITC_c)


EITC celing as array: 
[[ 487. 3250. 5372. 6044.]
 [ 496. 3305. 5460. 6143.]
 [ 503. 3359. 5548. 6242.]
 [ 506. 3373. 5572. 6269.]
 [ 510. 3400. 5616. 6318.]
 [ 519. 3461. 5716. 6431.]
 [ 519. 3461. 5716. 6431.]
 [ 519. 3461. 5716. 6431.]
 [ 519. 3461. 5716. 6431.]
 [ 519. 3461. 5716. 6431.]
 [ 519. 3461. 5716. 6431.]
 [ 519. 3461. 5716. 6431.]
 [ 519. 3461. 5716. 6431.]
 [ 519. 3461. 5716. 6431.]
 [ 519. 3461. 5716. 6431.]
 [ 519. 3461. 5716. 6431.]]


Note that 519, 3461, 5716, and 6431 have been set as the default value for year 2018 to 2028.

Indexed Parameters Intuition
-----------------------------------


The previous example shows how to extend parameter values along a given axis, like "year". However, what's really going on is that they are just repeated until the maximum year is reached. Now, it's time to index them at some specified rate.

To "grow" a parameter forward a year, we need to multiply it by one plus the rate at which it is expected to grow or its recorded growth rate. This is straightforward to do with a single dimensional value: just grab the previous value and multiply it by the inflation rate!

In [37]:
import taxcalc
import numpy as np

# use taxcalc inflation rates
rates = taxcalc.Policy().inflation_rates()

min_allowed_year = 2013
max_allowed_year = 2028
# max specified year
max_year = 2019

vals = [{"year": 2013 + i, "value": 1000 + i} for i in range(max_year - min_allowed_year + 1)]

print("rates", rates)
print("vals")
pprint(vals)

for ix, year in enumerate(range(max_year + 1, max_allowed_year + 1)):
    new_val = {
        "year": year,
        # previous value * inflation rate for current year!
        "value": np.round(vals[-1]["value"] * (1 + rates[max_year - min_allowed_year + ix]), 2)
    }
    vals.append(new_val)
    
print("result: ")
pprint(vals)

rates [0.0148, 0.0159, 0.0012, 0.0127, 0.0189, 0.0229, 0.0199, 0.0224, 0.0227, 0.0223, 0.0218, 0.0215, 0.0211, 0.021, 0.021, 0.0211]
vals
{'year': 2013, 'value': 1000}
{'year': 2014, 'value': 1001}
{'year': 2015, 'value': 1002}
{'year': 2016, 'value': 1003}
{'year': 2017, 'value': 1004}
{'year': 2018, 'value': 1005}
{'year': 2019, 'value': 1006}
result: 
{'year': 2013, 'value': 1000}
{'year': 2014, 'value': 1001}
{'year': 2015, 'value': 1002}
{'year': 2016, 'value': 1003}
{'year': 2017, 'value': 1004}
{'year': 2018, 'value': 1005}
{'year': 2019, 'value': 1006}
{'year': 2020, 'value': 1026.02}
{'year': 2021, 'value': 1049.0}
{'year': 2022, 'value': 1072.81}
{'year': 2023, 'value': 1096.73}
{'year': 2024, 'value': 1120.64}
{'year': 2025, 'value': 1144.73}
{'year': 2026, 'value': 1168.88}
{'year': 2027, 'value': 1193.43}
{'year': 2028, 'value': 1218.49}


Suppose, this parameter has another dimension, like marital status ("MARS" in Tax-Calculator terms). The previous value look up needs to account for the current value's MARS value, too. One approach is to think about the value as a two dimensional array like this:

```
vals = [
    [1000, 10000, 10000, 10000, 10000],
    [1001, 10001, 10001, 10001, 10001],
    [1002, 10002, 10002, 10002, 10002],
    [1003, 10003, 10003, 10003, 10003],
    [1004, 10004, 10004, 10004, 10004],
    [1005, 10005, 10005, 10005, 10005],
    [1006, 10006, 10006, 10006, 10006],
]
```

This array can be expaned by doing `vals.append(array_multiply(vals[-1] , inflation_rate))`. However, it is expensive for ParamTools to convert values with different shapes to arrays. I detail some of these issues in ParamTools issue [#12][1]. At the end of this doc, I also give an alternative (and much slower) solution that solves this problem with arrays.

So, let's get back to ParamTools' favorite data structure: a list of dictionaries. Here's some data with a MARS dimension.


[1]: https://github.com/PSLmodels/ParamTools/issues/12

In [39]:
MARS = ["single", "joint", "separate", "headhousehold", "widow"]

mars_vals = [{"year": 2013 + i, "MARS": status, "value": 1000 + i} 
             for i in range(max_year - min_allowed_year + 1) 
             for status in MARS]
pprint(mars_vals[:-10])
print("...")

{'year': 2013, 'MARS': 'single', 'value': 1000}
{'year': 2013, 'MARS': 'joint', 'value': 1000}
{'year': 2013, 'MARS': 'separate', 'value': 1000}
{'year': 2013, 'MARS': 'headhousehold', 'value': 1000}
{'year': 2013, 'MARS': 'widow', 'value': 1000}
{'year': 2014, 'MARS': 'single', 'value': 1001}
{'year': 2014, 'MARS': 'joint', 'value': 1001}
{'year': 2014, 'MARS': 'separate', 'value': 1001}
{'year': 2014, 'MARS': 'headhousehold', 'value': 1001}
{'year': 2014, 'MARS': 'widow', 'value': 1001}
{'year': 2015, 'MARS': 'single', 'value': 1002}
{'year': 2015, 'MARS': 'joint', 'value': 1002}
{'year': 2015, 'MARS': 'separate', 'value': 1002}
{'year': 2015, 'MARS': 'headhousehold', 'value': 1002}
{'year': 2015, 'MARS': 'widow', 'value': 1002}
{'year': 2016, 'MARS': 'single', 'value': 1003}
{'year': 2016, 'MARS': 'joint', 'value': 1003}
{'year': 2016, 'MARS': 'separate', 'value': 1003}
{'year': 2016, 'MARS': 'headhousehold', 'value': 1003}
{'year': 2016, 'MARS': 'widow', 'value': 1003}
{'year': 201

Before we get too bogged down in inflation rate details, let's figure out how to get the previous year's value, while taking into account the current value object's MARS value.

In [30]:
def get_mars_lookup(value_objects):
    """Return dictionary where the MARS values are the keys."""
    return {vo["MARS"]: {"value": vo["value"]} for vo in value_objects}

vals = [
    {'year': 2013, 'MARS': 'single', 'value': 'single val'},
    {'year': 2013, 'MARS': 'joint', 'value': 1000},
    {'year': 2013, 'MARS': 'separate', 'value': 1000},
    {'year': 2013, 'MARS': 'headhousehold', 'value': 1000},
    {'year': 2013, 'MARS': 'widow', 'value': 1000}
]
mars_lookup_2013 = get_mars_lookup(vals)

print("Look up the value for MARS=single: ", mars_lookup_2013["single"]["value"])

Look up the value for MARS=single:  single val


Perfect, now let's re-write the single-dimension example to work with the MARS parameters.

In [44]:
import taxcalc
import numpy as np

rates = taxcalc.Policy().inflation_rates()

min_allowed_year = 2013
max_allowed_year = 2028
# max specified year
max_year = 2019

MARS = ["single", "joint", "separate", "headhousehold", "widow"]

mars_vals = [{"year": 2013 + i, "MARS": status, "value": 1000 + i} 
             for i in range(max_year - min_allowed_year + 1) 
             for status in MARS]

for ix, year in enumerate(range(max_year + 1, max_allowed_year + 1)):
    prev_year = year - 1
    prev_year_vals = [val for val in mars_vals if val["year"] == prev_year]
    mars_lookup = get_mars_lookup(prev_year_vals)
    for status in MARS:
        new_val = {
            "year": year,
            "MARS": status,
            # previous value * inflation rate for current year!
            "value": np.round(mars_lookup[status]["value"] * (1 + rates[max_year - min_allowed_year + ix]), 2)
        }
        mars_vals.append(new_val)
    
print("result for a subsection of the values: ")
pprint(mars_vals[25:-25])
print("...")

result for a subsection of the values: 
{'year': 2018, 'MARS': 'single', 'value': 1005}
{'year': 2018, 'MARS': 'joint', 'value': 1005}
{'year': 2018, 'MARS': 'separate', 'value': 1005}
{'year': 2018, 'MARS': 'headhousehold', 'value': 1005}
{'year': 2018, 'MARS': 'widow', 'value': 1005}
{'year': 2019, 'MARS': 'single', 'value': 1006}
{'year': 2019, 'MARS': 'joint', 'value': 1006}
{'year': 2019, 'MARS': 'separate', 'value': 1006}
{'year': 2019, 'MARS': 'headhousehold', 'value': 1006}
{'year': 2019, 'MARS': 'widow', 'value': 1006}
{'year': 2020, 'MARS': 'single', 'value': 1026.02}
{'year': 2020, 'MARS': 'joint', 'value': 1026.02}
{'year': 2020, 'MARS': 'separate', 'value': 1026.02}
{'year': 2020, 'MARS': 'headhousehold', 'value': 1026.02}
{'year': 2020, 'MARS': 'widow', 'value': 1026.02}
{'year': 2021, 'MARS': 'single', 'value': 1049.0}
{'year': 2021, 'MARS': 'joint', 'value': 1049.0}
{'year': 2021, 'MARS': 'separate', 'value': 1049.0}
{'year': 2021, 'MARS': 'headhousehold', 'value': 1049

`IndexedParameters` Class
-----------------------------

Now that we have some intuition for how to solve this problem, let's tie it back into the `IndexedParameters` class. Below, we have simple class `IndexedParameters` with a gnarly `extend` method. I'll break it down and show that the previous section of has developed some intuition for what is going on below. Before we get to that, I'll prove that it works.

In [45]:
from collections import defaultdict

from marshmallow import Schema, fields
import numpy as np
import paramtools

import taxcalc


class IndexedParameters(paramtools.Parameters):
    def __init__(self):
        super().__init__()
        self.extend()
        self.array_first = True
        self.set_state()

    def extend(self):
        min_allowed_year = min(self._stateless_dim_mesh["year"])
        max_allowed_year = max(self._stateless_dim_mesh["year"])
        adjustment = defaultdict(list)
        param_data = self.specification(use_state=False, meta_data=True)
        
        # Compare to get_mars_lookup!
        def get_vo_lookup(vos, dims):
            qh = {}
            for vo in vos:
                qh[tuple(vo[d] for d in dims)] = vo["value"]
            return qh

        for param, data in param_data.items():
            max_year = max(map(lambda x: x["year"], data["value"]))
            if max_year == max_allowed_year:
                continue
            max_year = data["value"][-1]["year"]
            if max_year == max_allowed_year:
                continue
            # _get is an internal method for looking up parameter values.
            value_objects = self._get(param, True, year=max_year)
            if data["cpi_inflated"]:
                # preserve order by sorting the dimensions!
                dims_to_match = sorted(
                    [
                        dim_name
                        for dim_name in value_objects[0]
                        if dim_name not in ("year", "value")
                    ]
                )
                vo_lookup = get_vo_lookup(value_objects, dims_to_match)
                rates = self.indexing_rates(param)
                for ix, year in enumerate(
                    range(max_year + 1, max_allowed_year + 1)
                ):
                    for vo in value_objects:
                        dim_values = tuple(
                            vo[dim_name] for dim_name in dims_to_match
                        )
                        v = vo_lookup[dim_values] * (
                            1 + rates[max_year - min_allowed_year + ix]
                        )
                        v = np.round(v, 2) if v < 9e99 else 9e99
                        adjustment[param].append(
                            dict(vo, **{"year": year, "value": v})
                        )
                        vo_lookup[dim_values] = v
            else:
                for year in range(max_year, max_allowed_year + 1):
                    for vo in value_objects:
                        adjustment[param].append(dict(vo, **{"year": year}))
        self.adjust(adjustment)

    def indexing_rates(self, param=None):
        pass


`IndexedParameters` isn't meant to stand on its own. `TaxParams` must implement it before it can be used.

In [46]:
class TaxParams(IndexedParameters):
    schema = "schema.json"
    defaults = "defaults.json"
    field_map = {"compatible_data": fields.Nested(CompatibleDataSchema())}

    def __init__(self, *args, **kwargs):
        # Prepare the taxcalc inflation rates.
        growfactors = taxcalc.GrowFactors()
        self._inflation_rates = growfactors.price_inflation_rates(2013, 2028)
        self._apply_clp_cpi_offset(2028 - 2013 + 1)
        self._wage_growth_rates = growfactors.wage_growth_rates(2013, 2028)
        super().__init__(*args, **kwargs)

    def indexing_rates(self, param=None):
        """
        See taxcalc.Parameters.indexing_rates.
        """
        if param == "_SS_Earnings_c":
            return self._wage_growth_rates
        else:
            return self._inflation_rates

    def _apply_clp_cpi_offset(self, num_years):
        """
        See taxcalc.Policy._apply_clp_cpi_offset

        If you are curious about what's going on here, the
        cpi_offset parameter is an approximation for the chained
        cpi.
        """
        cpi_offset = [0.0, 0.0, 0.0, 0.0, -0.0025]
        if len(cpi_offset) < num_years:  # extrapolate last known value
            cpi_offset = cpi_offset + cpi_offset[-1:] * (
                num_years - len(cpi_offset)
            )
        for idx in range(0, num_years):
            infrate = round(self._inflation_rates[idx] + cpi_offset[idx], 6)
            self._inflation_rates[idx] = infrate


In [47]:
params = TaxParams()
# taxcalc abbreviates the standard deduction to "STD".
print("paramtools: ", params._STD, "\n")

pol = taxcalc.Policy()
print("taxcalc: ", pol._STD)



paramtools:  [[ 6100.   12200.    6100.    8950.   12200.  ]
 [ 6200.   12400.    6200.    9100.   12400.  ]
 [ 6300.   12600.    6300.    9250.   12600.  ]
 [ 6300.   12600.    6300.    9300.   12600.  ]
 [ 6350.   12700.    6350.    9350.   12700.  ]
 [12000.   24000.   12000.   18000.   24000.  ]
 [12274.8  24549.6  12274.8  18412.2  24549.6 ]
 [12519.07 25038.14 12519.07 18778.6  25038.14]
 [12799.5  25598.99 12799.5  19199.24 25598.99]
 [13090.04 26180.09 13090.04 19635.07 26180.09]
 [13381.95 26763.9  13381.95 20072.93 26763.9 ]
 [13673.68 27347.36 13673.68 20510.52 27347.36]
 [13967.66 27935.33 13967.66 20951.49 27935.33]
 [ 7690.   15380.    7690.   11323.   15380.  ]
 [ 7851.49 15702.98  7851.49 11560.78 15702.98]
 [ 8016.37 16032.74  8016.37 11803.56 16032.74]] 

taxcalc:  [[ 6100.   12200.    6100.    8950.   12200.  ]
 [ 6200.   12400.    6200.    9100.   12400.  ]
 [ 6300.   12600.    6300.    9250.   12600.  ]
 [ 6300.   12600.    6300.    9300.   12600.  ]
 [ 6350.   127

Let's confirm that the results are the same.

In [48]:
for param in params.specification():
    np.testing.assert_allclose(getattr(params, param), getattr(pol, param))

print("No errors were raised!")

No errors were raised!



Initialization
-----------------

All that's happening here is that the data is loaded using the `paramtools.Parameters` constructor, the data is extended with the `extend` method below, and then the instance is swapped to `array_first` mode.

```python
class IndexedParameters(paramtools.Parameters):
    def __init__(self):
        super().__init__()
        self.extend()
        self.array_first = True
        self.set_state()
```

Extend
---------------

This is very similar to the MARS parameter indexing example that we walked through above. The differences are that all parameters are being iterated over, some parameters need indexing while some do not, and parameters could have a marital status (MARS) dimension, earned income credit (EIC) dimension, itemized deduction type (idedtype) dimension, or no other dimension.

```python
    def extend(self):
        
```

`_stateless_dim_mesh` holds a "mesh" of data for each allowed dimension. For "year", it holds a list of allowed years from 2013 to 2018. `extend` uses it to determine for how many years each parameter needs to be extended.

```python
        min_allowed_year = min(self._stateless_dim_mesh["year"])
        max_allowed_year = max(self._stateless_dim_mesh["year"])
        adjustment = defaultdict(list)
        param_data = self.specification(use_state=False, meta_data=True)
 ```
 
 `get_vo_lookup` is just a generalization of `get_mars_lookup`. It takes a list of value objects for a given year and creates a dictionary (or hash table) mapping the dimension values of a parameter to its value.
 
 ```python
        # Compare to get_mars_lookup!
        def get_vo_lookup(vos, dims):
            qh = {}
            for vo in vos:
                qh[tuple(vo[d] for d in dims)] = vo["value"]
            return qh
```


```python
        for param, data in param_data.items():
            # Find the maximum specified year.
            max_year = max(map(lambda x: x["year"], data["value"]))
            if max_year == max_allowed_year:
                # No need to do anything if the entire window is already 
                # specified.
                continue
```

`_get` is an internal method for looking up parameter values. It is used internally by for all 
data look ups. It's use here makes a good case for adding it back to the public API.

```python
            value_objects = self._get(param, True, year=max_year)
```

Only index the values that need to be indexed. Tax-Calculator uses `cpi_inflated` to indicate whether a value
needs to be inflated or not. 

```python
            if data["cpi_inflated"]:
```

The dimensions need to be sorted by their name in `dims_to_match`. Since the look up keys are created as a tuple of values, the order of the values needs to be consitent. If the values are inserted in a random order, then tuples that should match may appear to be different because the order is wrong.
```python
                # preserve order by sorting the dimensions!
                dims_to_match = sorted(
                    [
                        dim_name
                        for dim_name in value_objects[0]
                        if dim_name not in ("year", "value")
                    ]
                )
                vo_lookup = get_vo_lookup(value_objects, dims_to_match)
                rates = self.indexing_rates(param)
```

Iterate from the year following the max specified year to the max allowed year. Then, for each value object specified in `year`, create its lookup key using the sorted names in `dims_to_match`, lookup its value in the previous year, and multiply by the previous year's inflation rate. 

```python
                for ix, year in enumerate(
                    range(max_year + 1, max_allowed_year + 1)
                ):
                    for vo in value_objects:
                        dim_values = tuple(
                            vo[dim_name] for dim_name in dims_to_match
                        )
                        v = vo_lookup[dim_values] * (
                            1 + rates[max_year - min_allowed_year + ix]
                        )
                        # Tax-Calculator rounds to two decimal places.
                        # It also uses 9e99 as an infity type value for
                        # some variables.
                        v = np.round(v, 2) if v < 9e99 else 9e99
                        adjustment[param].append(
                            dict(vo, **{"year": year, "value": v})
                        )
                        vo_lookup[dim_values] = v
```

```python
            else:
                for year in range(max_year, max_allowed_year + 1):
                    for vo in value_objects:
                        adjustment[param].append(dict(vo, **{"year": year}))
```

`extend` essentially creats one massive adjustment containing all of the new extended and indexed values. `adjust` updates the current values with these extended ones.
```python
        self.adjust(adjustment)

    def indexing_rates(self, param=None):
        pass
```